In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("duration-prediction")

2023/07/20 14:50:52 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/07/20 14:50:52 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/mlops-ws/notebooks/mlruns/1', creation_time=1689844853410, experiment_id='1', last_update_time=1689844853410, lifecycle_stage='active', name='duration-prediction', tags={}>

In [ ]:
df = pd.read_parquet('./data/green_tripdata_2021-01.parquet')

In [ ]:
df

In [ ]:
#df = df[df.trip_type ==2]

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
#df = df[df.trip_type == 2]

In [ ]:
df

In [ ]:
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime

In [ ]:
td = df.duration.iloc[0]

In [ ]:
td.total_seconds()/60

In [ ]:
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

In [ ]:
df.duration

In [ ]:
#df = df[df.trip_type == 2]

In [ ]:
sns.displot(df.duration)

In [ ]:
df.duration.describe(percentiles=[0.95,0.98,0.99])

In [ ]:
((df.duration >= 1) & (df.duration <=60)).mean()

In [ ]:
df = df[(df.duration >=1) & (df.duration <=60)]
df

In [ ]:
categorical = ["PULocationID","DOLocationID"]

In [ ]:
numerical = ['trip_distance']

In [ ]:
df[categorical].dtypes

In [ ]:
df[categorical].astype(str).dtypes

In [ ]:
df[categorical] = df[categorical].astype(str)

In [ ]:
df.dtypes

In [ ]:
dv = DictVectorizer()

In [ ]:
traindict = df[categorical + numerical].to_dict(orient='records')
traindict

In [ ]:
X_train = dv.fit_transform(traindict)

In [ ]:
X_train

In [ ]:
dv.feature_names_

In [ ]:
target = "duration"

In [ ]:
y_train = df[target].values

In [ ]:
y_train

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_train)

In [ ]:
sns.distplot(y_pred,label="prediction")
sns.distplot(y_train,label="train")

plt.legend()

In [ ]:
mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
# data prep function:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    
    df = df[(df.duration >=1) & (df.duration <=60)]
    
    categorical = ["PULocationID","DOLocationID"]
    numerical = ['trip_distance']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [ ]:
len(df_train),len(df_val)

In [ ]:
categorical = ["PULocationID","DOLocationID"]
numerical = ['trip_distance']

dv = DictVectorizer()

traindict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(traindict)
X_train

valdict = df_val[categorical + numerical].to_dict(orient='records')
X_val  = dv.transform(valdict) 
X_val


target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

    

In [ ]:
# other model with lasso regression and ridge

In [ ]:
la = Lasso(alpha=0.01)
la.fit(X_train,y_train)
y_pred = la.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
rg = Ridge(alpha=0.001)
rg.fit(X_train,y_train)
y_pred = rg.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
# to to change features
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ["PU_DO"]#["PULocationID","DOLocationID"]
numerical = ['trip_distance']

dv = DictVectorizer()

traindict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(traindict)
X_train

valdict = df_val[categorical + numerical].to_dict(orient='records')
X_val  = dv.transform(valdict) 
X_val


target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
with open("models/lin_reg.bin","wb") as f_out:
    pickle.dump((dv,lr),f_out)

In [ ]:
la = Lasso(alpha=0.0001)
la.fit(X_train,y_train)
y_pred = la.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

In [ ]:
rg = Ridge(alpha=0.001)
rg.fit(X_train,y_train)
y_pred = rg.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)